In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.colors as colors
import matplotlib.pyplot as plt

def getHeatMap(newdataf,month,filepath):
    #Filtraggio per mese
    newdf=newdataf[newdataf['hours'].str.contains(month)]
    #Conta numero di osservazioni in ogni ora
    newdf=newdf.groupby(['hours'],sort=False,as_index=False)[['count']].count()
    #Estrai giorno e ora
    newdf['day']=newdf['hours'].str[:2]
    newdf['h']=newdf['hours'].str[-3:-1]
    #Crea nuovo daframe per heatmap
    f=pd.DataFrame(columns=['h','count'])
    f['day']=newdf['day']
    f['h']=newdf['h'].astype(int)
    f['count']=newdf['count'].astype(int)
    #Crea una pivot table
    piv = pd.pivot_table(f, values="count",index=["h"], columns=["day"], fill_value=0)
    # This dictionary defines the colormap
    cdict = {'red':  ((0.0, 0.8, 0.8),   # no red at 0
                  (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
                  (1.0, 0.0, 0.0)),  # set to 0.8 so its not too bright at 1

        'green': ((0.0, 0.0, 0.0),   # set to 0.8 so its not too bright at 0
                  (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
                  (1.0, 0.8, 0.8)),  # no green at 1

        'blue':  ((0.0, 0.0, 0.0),   # no blue at 0
                  (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
                  (1.0, 0.0, 0.0))   # no blue at 1
       }
    GnRd = colors.LinearSegmentedColormap('GnRd', cdict)
    ax = sns.heatmap(piv, square=True,linewidths=0.8,cmap=GnRd)
    figure = ax.get_figure() 
    heatDir=os.path.join(os.path.abspath('..'),"heatmap"+filepath[:-4])
    if not os.path.exists(heatDir):
        os.mkdir(heatDir)
    heatmapPath=os.path.join(heatDir,month+'.png')
    figure.savefig(heatmapPath, dpi=400)
    plt.clf()

def process(chunck,filepath):
    df=chunck.loc[:, ('timestamp', 'power')]
    # generiamo la data nel formato corretto a partire dal timestamp unix
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms').dt.tz_localize("GMT").dt.tz_convert('Europe/Rome')
    # arrotondiamo da ms al secondo pi� vicino
    df['date'] = df['date'].dt.round('1s')
    df = df[['date', 'power']]
    # crea una time series settando la data come index
    df = df.set_index('date')
    #Raggruppa per data ed effettua la media dei power
    df=df.groupby('date')[['power']].mean()
    #Aggiungi una colonna contenente  data e ora per heatmap
    df['hours']=df.index.to_series(keep_tz=True).dt.strftime("%d-%b-%Y (%H)")
    df['day']=df.index.to_series(keep_tz=True).dt.strftime("%d").str.lstrip("0").replace(" 0", " ")
    df['count']=0
    monthList=['Jan','Feb','Mar','Apr','May','Jun']
    for month in monthList:
        newdf=df[df['hours'].str.contains(month)]
        getHeatMap(newdf,month,filepath) 

In [11]:
DIRPATH="\Dati puntuali EB1 Gennaio-Giugno 2019"
FILEPATHLIST={"\Dati puntuali Frigo EB1 2019.csv","\Dati puntuali Lavastoviglie EB1 2019.csv","\Dati puntuali Lavatrice EB1 2019.csv","\Dati puntuali TV EB1 2019.csv"}

for filepath in FILEPATHLIST:
    dataPath=os.path.join(os.path.abspath('..'),"data"+DIRPATH+filepath)
    #for chunck in pd.read_csv(dataPath, sep=';',chunksize=10000):
    process( pd.read_csv(dataPath, sep=';'),filepath)

<Figure size 432x288 with 0 Axes>

In [5]:
'''
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                       'B': ['B0', 'B1', 'B2', 'B3'],
                       'C': ['C0', 'C1', 'C2', 'C3'],
                   'D': ['D0', 'D1', 'D2', 'D3']})
df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                       'B': ['B4', 'B5', 'B6', 'B7'],
                     'C': ['C4', 'C5', 'C6', 'C7'],
                       'D': ['D4', 'D5', 'D6', 'D7']})
f=pd.DataFrame(columns=['A','B','C','D'])
f
frames = [df1,f]
pd.concat(frames)
'''

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
